# Projects with Python



## Example 1: `USA.gov` Data From `Bitly`
In 2011, URL shortening service `Bitly` partnered with the US government website
`USA.gov` to provide a feed of anonymous data gathered from users who shorten links
ending with `.gov` or `.mil`. The data is gotten from @McK2017.

The data file can be downloaded from [here](assests/datasets/example.txt). The file is mostly in JSON. It can be converted into a DataFrame by the following code.

In [ ]:
import pandas as pd
import numpy as np
import json
path = 'assests/datasets/example.txt'
df = pd.DataFrame([json.loads(line) for line in open(path)])

We mainly use `tz` and `a` columns. So let us clean it.

In [ ]:
#| warning: false
df['tz'] = df['tz'].fillna('Missing')
df['tz'][df['tz'] == ''] = 'Unknown'
df['a'] = df['a'].fillna('Missing')
df['a'][df['a'] == ''] = 'Unknown'

We first want to extract the timezone infomation from it. The timezone info is in the column `tz`.

In [ ]:
tzone = df['tz']
tvc = tzone.value_counts()
tvc

After cleaning data, we would like to visulize the value counts.

In [ ]:
import seaborn as sns
sns.barplot(x=tvc[:10].values, y=tvc[:10].index)

We then would like to extract information from the column `a`. This column is about the agent of the connection. The important info is the part before the space `' '`.


In [ ]:
agent = df['a']
agent = agent.str.split(' ').str[0]
avc = agent.value_counts()
avc[:10]

Now let us assume that, if `Windows` appears in column `a` the user is using `Windows` os, if not then not. In this case, the os can be detected by the following code.

In [ ]:
df['os'] = np.where(df['a'].str.contains('Windows'), 'Windows', 'Not Windows')

Now we can make a bar plot about the counts based on `os` and `timezone`.

In [ ]:
tz_os_counts = df.groupby(['tz', 'os']).size().unstack().fillna(0)
tz_os_counts.head()

We then turn it into a DataFrame using the `.stack()`, `.unstack()` tricks.

In [ ]:
tovc = tz_os_counts.stack()[tz_os_counts.sum(axis=1).nlargest(10).index]
tovc.name = 'count'
dftovc = pd.DataFrame(tovc).reset_index()

Finally we may draw the bar plot.

In [ ]:
sns.barplot(x='count', y='tz', hue='os', data=dftovc)

## Example 2: US Baby Names 1880–2010
The United States Social Security Administration (SSA) has made available data on the frequency of baby names from 1880 through the present. Hadley Wickham, an author of several popular R packages, has often made use of this dataset in illustrating data manipulation in `R`. The dataset can be downloaded from [here](assests/datasets/babynames.zip) as a zip file. Please unzip it and put it in your working folder.

In the folder there are 131 `.txt` files. The naming scheme is `yob` + the year. Each file contains 3 columns: `name`, `gender`, and `counts`. We would like to add a column `year`, and combine all files into a single DataFrame. In our example, the year is from 1880 to 2010.


In [ ]:
#| eval: false
import pandas as pd

path = 'assests/datasets/babynames/'
dflist = list()
for year in range(1880, 2011):
    filename = path + 'yob' + str(year) + '.txt'
    df = pd.read_csv(filename, names=['name', 'gender', 'counts'])
    df['year'] = year
    dflist.append(df)
df = pd.concat(dflist, ignore_index=True)

We can plot the total births by sex and year.

In [ ]:
#| eval: false
import seaborn as sns

sns.relplot(data=df.groupby(['gender', 'year']).sum().reset_index(),
            x='year', y='counts', hue='gender', kind='line')

In [ ]:
#| echo: false
import seaborn as sns
sns.set_theme()
dfgy = pd.read_csv('assests/datasets/gy.csv')
sns.relplot(data=dfgy,
            x='year', y='counts', hue='gender', kind='line')

For further analysis, we would like to compute the proportions of each name relative to the total number of births per year per gender. 

In [ ]:
#| eval: false
def add_prop(group):
    group['prop'] = group.counts / group.counts.sum()
    return group

df = df.groupby(['gender', 'year']).apply(add_prop)
df.head()

In [ ]:
#| echo: false
babydf5 = pd.read_csv('assests/datasets/babydf5.csv')
babydf5

Now we would like to keep the first 100 names in each year, and save it as a new DataFrame `top100`.

In [ ]:
#| eval: false
top100 = (
    df.groupby(['year', 'gender'])
    .apply(lambda x: df.loc[x['counts'].nlargest(100).index])
    .drop(columns=['year', 'gender'])
    .reset_index()
    .drop(columns='level_2')
)
top100.head()

In [ ]:
#| echo: false
dftop100 = pd.read_csv('assests/datasets/baby100.csv')
dftop100

Note that `level_2` is related to the original index after `reset_index()`. That's why we don't need it here.

Now we would like to draw the trend of some names.

In [ ]:
#| eval: false
namelist = ['John', 'Harry', 'Mary']
sns.relplot(data=top100[top100['name'].isin(namelist)],
            x='year', y='counts', hue='name', kind='line')

In [ ]:
#| echo: false
babynamelist = pd.read_csv('assests/datasets/babynamelist.csv')
sns.relplot(data=babynamelist,
            x='year', y='counts', hue='name', kind='line')

Now we would like to analyze the ending of names.

In [ ]:
#| eval: false
df['ending'] = df['name'].str[-1]
endingcount = df.groupby(['gender', 'year', 'ending']).sum().reset_index()

We would like to draw barplots to show the distributions in year 1910, 1960 and 2010.

In [ ]:
#| eval: false
certainyear = endingcount[endingcount['year'].isin([1910, 1960, 2010])]
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 1, figsize=(10,7))
sns.barplot(data=certainyear[endingcount['gender']=='M'],
            x='ending', y='prop', hue='year', ax=axs[0])
sns.barplot(data=certainyear[endingcount['gender']=='F'],
            x='ending', y='prop', hue='year', ax=axs[1]).legend_.remove()

In [ ]:
#| echo: false
babyyear = pd.read_csv('assests/datasets/babyyear.csv')
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 1, figsize=(10,7))
sns.barplot(data=babyyear[babyyear['gender']=='M'],
            x='ending', y='prop', hue='year', ax=axs[0])
sns.barplot(data=babyyear[babyyear['gender']=='F'],
            x='ending', y='prop', hue='year', ax=axs[1]).legend_.remove()

We would also like to draw the line plot to show the trending of certain letters through years.

In [ ]:
#| eval: false
sns.relplot(data=endingcount[endingcount.ending.isin(['d', 'n', 'y'])],
            x='year', y='prop', hue='ending', kind='line')

In [ ]:
#| echo: false
babyending = pd.read_csv('assests/datasets/babyending.csv')
sns.relplot(data=babyending[babyending.ending.isin(['d', 'n', 'y'])],
            x='year', y='prop', hue='ending', kind='line')

## Exercises

::: {#exr-}
Please use the baby name dataset. We would like to consider the diversity of the names. Please compute the number of popular names in top 50% for each year each gender. Draw a line plot to show the trend and discuss the result.
:::
